# Export data from User Interface (UI) to analysis workspace
This is a tutorial notebook that walks through the process of exporting selected data from the *NHLBI BioData Catalyst® (BDC) Powered by PIC-SURE* User Interface, or UI, into an analysis workspace. This is done using the *BDC Powered by PIC-SURE (BDC-PIC-SURE)* Application Programming Interface, or API.

------- 
 
## Introduction to exporting data into an analysis workspace with PIC-SURE

Two things are needed to export data into an analysis workspace:
1. Personalized access token: a user-specific token that tells PIC-SURE which studies a user is authorized to access
2. Query ID: a token that describes the specific query that was built in the UI, for example, if a user has selected females with body mass index between 18 and 30 from the ARIC study

Using these two components, the API can be used to export the selected data into the analysis workspace (in this case, where this Jupyter Notebook is being run). 

## Step 1: Getting your user-specific security token
**Before running this notebook, please be sure to review the "Get your security token" documentation, which exists in the [`README.md` file](../README.md). It explains how to get a security token, which is mandatory to use the PIC-SURE API.**

To set up your token file, be sure to run the [`Workspace_setup.ipynb` file](./Workspace_setup.ipynb).

## Step 2: Setting up your notebook

### Pre-requisites for the notebook
* python 3.6 or later
* pip python package manager, already available in most systems with a python interpreter installed (link to pip)

### Install packages to connect to the PIC-SURE API
The first step to using the PIC-SURE API is to install the packages needed. The following code installs the PIC-SURE API components from GitHub, specifically:
* PIC-SURE Client
* PIC-SURE Adapter

**Note that if you are using the dedicated PIC-SURE environment within the *BDC Powered by Seven Bridges (BDC-Seven Bridges)* platform, the necessary packages have already been installed.**

*Troubleshooting: if you encounter an error while installing packages or connecting to a PIC-SURE resource, try restarting your kernel and trying again.*

In [ ]:
### Uncomment this code if you are not using the PIC-SURE environment in *BDC-Seven Bridges*, or if you do not have all the necessary dependencies installed.
#install.packages("devtools")

In [ ]:
Sys.setenv(TAR = "/bin/tar")
options(unzip = "internal")
devtools::install_github("hms-dbmi/pic-sure-r-adapter-hpds", ref="main", force=T, quiet=FALSE)
library(dplyr)

## Connecting to a PIC-SURE resource
The following is required to get access to data through the PIC-SURE API: 
- Network URL
- User-specific security token

The following code specifies the network URL as the *BDC-PIC-SURE* URL and references the user-specific token saved as `token.txt`.

If you have not already retrieved your user-specific token, please refer to the "Get your security token" section of the `README.md` file and the `Workspace_setup.ipynb` file.

In [ ]:
PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
token_file <- "token.txt"
token <- scan(token_file, what = "character")
session <- picsure::bdc.initializeSession(PICSURE_network_URL, token)
session <- picsure::bdc.setResource(session = session,  resourceName = "AUTH")

## Step 3: Export data from a query built in the PIC-SURE UI using the Query ID

You are able to retrieve the results of a query that you have previously built using the [PIC-SURE UI](https://picsure.biodatacatalyst.nhlbi.nih.gov/). After you have built your query and filtered to your cohort of interest, click the "Prepare Data for Analysis" button. This will allow you to copy your query ID and bring it into a Jupyter notebook.

In [ ]:
# To run this using your notebook you must replace it with the ID value of a query that you have run.
queryID <- 'paste your query ID here'



In [ ]:
# save the data as a dataframe
results <- picsure::getResultByQueryUUID(session, queryID)

# view the first few records
head(results)



The data has now been exported as a dataframe saved as `results` and is ready for analysis.

## Bonus: Edit from a query built in the PIC-SURE UI using the Query ID
You can now use the PIC-SURE API to edit queries that were built in the PIC-SURE UI. To do this, follow the same steps outlined above to build a query, package the data, and retrieve a Query ID. 

In [ ]:
# To run this using your notebook you must replace it with the ID value of a query that you have run.
queryID <- 'paste your query ID here'
query <- picsure::bdc.getQueryByUUID(session, queryID)




You can use the following code to view the filters and variables added for export. There are several different fields shown in this output.

| Field | Meaning | Output | Example |
|--------|-------------------|-------|-------|
| query\$fields | All variables included in the list (no record subsetting) | Automatically generated PIC-SURE variables; variables added to export | `\\_Topmed Study Accession with Subject ID\\`, <br />`\\_Parent Study Accession with Subject ID\\`, <br />`\\phs000820\\pht004333\\phv00219059\\sampleID\\` |
| query\$requiredFields | All variables; only records that do not contain null values for input variables | Variables filters to all values (such as selecting both "Is a tumor" and "Is not a tumor" for a "Tumor status" variable) | `\\phs000820\\pht004333\\phv00219063\\is_tumor\\` |
| query\$anyRecordOf | All variables; only records that contain at least one non-null value for input variables | Variables added from the Dataset modal | `\\phs000820\\pht004332\\phv00219058\\AfibYes\\` |
| query\$categoryFilters | Only records that match filter criteria for added categorical variables | Automatically generated PIC-SURE variables; variables that have been filtered (such as selecting only "Male" for a "Sex" variable | `$\\_consents\\ \| ['phs000820.c1']`,<br /> `categorical \| '\\phs000820\\pht004332\\phv00219057\\sex\\' \| 'Male'`|
| query\$numericFilters | Only records that match filter criteria for added numeric variables | Variables that have been filtered (such as selecting participants between 30 and 70 years old | `\\phs000820\\pht004332\\phv00219056\\age\\": "min": "30", "max": "70"`|

In [ ]:
# View all of the selected "fields" added to the query
query$fields


In [ ]:
# View all of the "requiredFields" added to the query
query$requiredFields



In [ ]:
# View all of the "anyRecordOf" fields added to the query
query$anyRecordOf



In [ ]:
# View all of the "categoryFilters" fields added to the query
query$categoryFilters

# Note - this includes a PIC-SURE required field called "\\_consents\\", which informs which studies and consent codes you are authorized to access. For more information about this field, view the "1_PICSURE_API_101" notebook.

In [ ]:
# View all of the "numericFilters" fields added to the query
query$numericFilters



Now that the query has been loaded, you can use the PIC-SURE API code to add new, edit existing, or delete clauses. 

There are several methods that can be used to build a query, which can be specified by the `type` argument in the `picsure::addClause` function. 

| type | functionality |
|--------|-------------------|
| type = 'SELECT' | return all variables included in the list (no record subsetting)|
| type = 'REQUIRE' | return all variables included in the list; only records that do not contain null values for input variables |
| type = 'ANYOF' | return all variables included in the list; only records that contain at least one non-null value for input variables |
| type = 'FILTER' | return input variable; only records that match filter criteria |


Additionally, the `picsure::deleteClause` function can be used to delete fields of the query. 

*Note: Please review the `1_PICSURE_API_101` notebook for more information and detailed examples about how to add to a query.*

In [ ]:
# EXAMPLE CODE
# Note: This code may not work with your query and is only intended to show how to set up code. Please adjust to your query and research purposes.

# Let's say we have a "Gender of participant" filtering to "Male" but want to also add "Female". We could accomplish this using the following code:
query <- picsure::addClause(query = query, 
                             keys = "\\phs000820\\pht004332\\phv00219057\\sex\\",
                             type = "FILTER",
                             categories = list("Male", "Female"))

# Similarly, we can edit an existing age range of 30 to 70 to be 35 to 75.
query <- picsure::addClause(query = query, 
                             keys = '\\phs000820\\pht004332\\phv00219056\\age\\',
                             type = "FILTER", 
                             min = 35,
                             max = 75)

# We can also delete variables and filters from the query altogether.
query <- picsure::deleteClause(query = query, '\\phs000820\\pht004333\\phv00219063\\is_tumor\\')